# HV19.12 back to basic

We get an exe which seems to be a VisualBasic application:

In [1]:
!strings BackToBasic.exe | grep __vba

__vbaFreeVar
__vbaVarForNext
__vbaStrVarVal
__vbaVarXor
__vbaI4Var
__vbaVarAdd
__vbaVarSub
__vbaVarForInit
__vbaLenVar
__vbaVarTstEq
__vbaFreeVarList
__vbaVarCmpEq
__vbaVarAnd
__vbaBoolVarNull
__vbaVarMove
__vbaFreeObjList
__vbaFreeStr
__vbaLenBstr
__vbaFreeObj
__vbaHresultCheckObj
__vbaObjSet
__vbaVarSub
__vbaVarMove
__vbaFreeVar
__vbaLenBstr
__vbaFreeVarList
__vbaFreeObjList
__vbaHresultCheckObj
__vbaLenVar
__vbaVarXor
__vbaVarForInit
__vbaObjSet
__vbaBoolVarNull
__vbaChkstk
__vbaVarTstEq
__vbaVarAnd
__vbaExceptHandler
__vbaFPException
__vbaStrVarVal
__vbaI4Var
__vbaVarCmpEq
__vbaVarAdd
__vbaVarForNext
__vbaFreeObj
__vbaFreeStr


If we run it on Windows, we get:

![Screenshot](./basic.png)

So the big question was how to get that into some readable format. At the point I solved this challenge, I hadn't used Ghidra/IDA yet (I later did on day 16). I also wasn't sure if it was the right tool for this job, so I looked for VBA disassemblers.

I found [VB Decompiler](https://www.vb-decompiler.org/) and [VBReFormer](https://qiil.io/) and used them to analyze the function called when the entered text changed. Unfortunately, all the free versions could produce was some 600 lines of disassembly.

At that point, I also found the string `"6klzic<=bPBtdvff'y\x7fFI~on//N"` (not a literal `\x7f`, but an ASCII DEL). This will be important for alter

A big part of the disassembly looked like this:

```x86asm
loc_004020B5: mov esi, [00401054h] ; rtcMidCharVar
loc_004020BB: lea eax, var_5C
loc_004020BE: push eax
loc_004020BF: lea ecx, var_34
loc_004020C2: push 00000001h
loc_004020C4: lea edx, var_6C
loc_004020C7: mov ebx, 00000002h
loc_004020CC: push ecx
loc_004020CD: push edx
loc_004020CE: mov var_54, 00000001h
loc_004020D5: mov var_5C, ebx
loc_004020D8: call rtcMidCharVar
```

In this snippet, the following happens:

- `var_54` is set to 1
- `var_5C` is set to 2 (via `ebx` which was set earlier)
- A call to the [`Mid` Visual Basic function](https://docs.microsoft.com/en-us/office/vba/language/reference/user-interface-help/mid-function)

To understand how the VisualBasic calls worked, I mainly used a [weirdly formatted blogpost](https://hvoidcode.wordpress.com/2016/02/06/vb-function-description-for-reversing/) and the [Chinese original](https://blog.csdn.net/weixin_30736301/article/details/95964460). Here's an example, based on the same code, but shortened and rearranged:

```x86asm
lea eax, var_5C
push eax

push 00000001h

lea ecx, var_34
push ecx

lea edx, var_6C
push edx

call rtcMidCharVar
```

The calling convention seems to be so that this corresponds to:

```basic
var_6C = Mid(var_34, 1, var_5C)
```

I basically did that for some six hours until I had an idea what the application was doing. Two things made this much more difficult:

- At some points where I was supposed to see some data, I only saw 0x8008 (same when looking at the file in Ghidra as part of writing this writeup)
- Some arguments in the disassembly seemed to be wrong. For example, for a (very important) `for`-loop which starts with 6, `var_164` is set to 6 (and never used), but then `__vbaVarForInit` is called with `var_6C`, `var_16C`, `var_1E4`, `var_1D4` and `var_24`. The VBReFormer tool ended up showing me `For var_24 = var_25 To #NOT SUPPORTED# Step 1` but not showing me `var_25` and `var_25` anywhere.

Because of all those issues, I needed a hint or two from someone who already solved the challenge.

After an excruciating six hours, I had an idea of what was happening - here is a rough Python reimplementation (trying to more or less keep the original structure, so not using Python idioms):

In [19]:
DATA = b"6klzic<=bPBtdvff'y\x7fFI~on//N"

def check(inp):
    mid1 = inp[0] # 004020B5
    mid2 = inp[1] # 004020DA
    mid3 = inp[2] # 00402113
    mid4 = inp[3] # 0040214D
    
    ok = mid1 == 'H'  # 00402187
    
    ok2 = mid2 == 'V' # 004021B0
    ok &= ok2         # 004021CC
    
    ok3 = mid3 == '1' # 004021DA
    ok &= ok3         # 004021F6
    
    ok4 = mid4 == '9' # 00402204
    ok &= ok4         # 00402220
    
    assert ok
    
    length = len(inp)    # 00402286
    assert length == 33  # 004022A8
    
    for i in range(6, 33):  # 004022BF to 0040232D
        assert ord(inp[i-1]) ^ i == DATA[i-6]  # 00402339 to 004023C0

So we can get the flag based on that weird string we found initially:

In [20]:
print('HV19{' + ''.join(chr(k ^ i) for i, k in enumerate(DATA, start=6)) + '}')

HV19{0ldsch00l_Revers1ng_Sess10n}
